In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import pyarrow
import csv

# KYC Preprocessing

The purpose of this notebook is to add occupation-specific indicators to the KYC data. Indicators are binary flags representing whether or not a client's occupation is part of a specific subset of occupations. These subsets were determined using Fintrac and financial crime recommendations. Mainly:

Per the [Fintrac Operational Alert](https://fintrac-canafe.canada.ca/intel/operation/oai-wildlife-eng), one indicator is "An individual is the owner, operator, employee or associated with an industry that could be used to facilitate illegal wildlife trade (e.g., import/export of goods, fisheries wholesaler, pet store, freight company, animal control)." (**G**)

The [Financial Crime Academy](https://financialcrimeacademy.org/wildlife-trade-risk-indicators-financial/?fbclid=IwAR1XSw09Vtl4mjOOQj_eTFuqZ_GKqM-SPsCJwQKcyFb-XWU4O6nO8zBo3JU) adds that another "indicator relates to activity involving politically exposed persons and wealthy businessmen/women, particularly those with environmental, game, or forestry oversight or environmental or wildlife-related businesses." (**G**)

We have therefore come up with the following indicators based on KYC occupation data:
- `occ_wealth` 
    - binary 
    - 1 if the client's occupation involves frequent exposure to wealthy people, 0 otherwise.
- `occ_animal` 
    - binary 
    - 1 if the client's occupation involves working with animals, 0 otherwise.
- `occ_int` 
    - binary 
    - 1 if the client works international trade, 0 otherwise.
- `occ_shipping`
    - binary
    - 1 if the client works in shipping/postal/cargo services
    - *there are no examples of this in our data*

## Merging Occupation and Customer Data

In [2]:
# Adding the occupation-based indicators to raw data
datapth = Path('../data/')
raw = pd.read_csv(datapth / 'raw' / 'kyc.csv')
occ = pd.read_csv(datapth / 'processed' / 'occupation_list.csv')

merged = raw.merge(occ, 'left', on='occupation')

# Re-order columns
cols = list(merged.columns)
cols.append(cols.pop(cols.index('label')))
merged = merged[cols]

# Export
display(merged.head(3))
merged.to_parquet(datapth / 'processed' / 'kyc.parquet', index=False)

,name,gender,occupation,age,tenure,cust_id,occ_wealth,occ_animal,occ_int,label
0,JENNIFER WELLS,female,Architect,45.0,13.0,CUST82758793,0,0,0,0
1,ANTHONY ADAMS,male,Musician,52.0,8.0,CUST69248708,1,0,0,0
2,DENISE LEWIS,female,Jewelry Dealer,43.0,11.0,CUST67222818,1,0,1,0


## Merging KYC and Transactional Data

In [3]:
kyc_df = merged.copy()
kyc_df = kyc_df.drop(['occupation', 'gender', 'name', 'age', 'tenure'], axis=1)

# Wire Transfer
wire_df = pd.read_csv(datapth / 'raw' / 'wire.csv')
wire_df = wire_df.merge(kyc_df.add_suffix('_receiver', axis=1), on='cust_id_receiver', how='left')
wire_df = wire_df.merge(kyc_df.add_suffix('_sender', axis=1), on='cust_id_sender', how='left')
wire_df.to_parquet(datapth / 'processed' / 'wire.parquet', index=False)
display(wire_df.sample(3))

# Emt
emt_df = pd.read_csv(datapth / 'raw' / 'emt.csv')
emt_df = emt_df.merge(kyc_df.add_suffix('_receiver', axis=1), on='cust_id_receiver', how='left')
emt_df = emt_df.merge(kyc_df.add_suffix('_sender', axis=1), on='cust_id_sender', how='left')
emt_df.to_parquet(datapth / 'processed' / 'emt.parquet', index=False)
display(emt_df.sample(3))

# Cash
cash_df = pd.read_csv(datapth / 'raw' / 'cash.csv')
cash_df = cash_df.merge(kyc_df, on='cust_id', how='left')
cash_df.to_parquet(datapth / 'processed' / 'cash.parquet', index=False)
display(cash_df.sample(3))

,cust_id_sender,cust_id_receiver,name_sender,name_receiver,trxn_value,country_sender,country_receiver,trxn_id,occ_wealth_receiver,occ_animal_receiver,occ_int_receiver,label_receiver,occ_wealth_sender,occ_animal_sender,occ_int_sender,label_sender
56303,CUST90537644,EXTERNAL785812,JOSHUA SANDERS,MELISSA ALVAREZ,7527.0,CA,CA,WSYU34655095,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0
4561,CUST13764193,CUST20521460,DR.STEPHANIE MURPHY,ANIKA SINGHAL,2200.0,CA,CA,LOOV76314955,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28961,EXTERNAL469571,CUST93115653,VANESSA BERNARD,RÉMY DELISLE-LEBLANC,7370.5,CA,CA,LLDI48962412,1.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN


,cust_id_sender,cust_id_receiver,name_sender,name_receiver,trxn_message,emt_value,trxn_id,occ_wealth_receiver,occ_animal_receiver,occ_int_receiver,label_receiver,occ_wealth_sender,occ_animal_sender,occ_int_sender,label_sender
85822,CUST56549444,CUST79296515,MATTHEW LE,DR.JULIA SHEPHERD,NaN,916.5,XOTZ77158613,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
340582,CUST11955232,EXTERNAL308306,MICHAEL HARRIS,SHAWN SMITH,NaN,10.0,VIKZ97796758,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
288662,EXTERNAL446072,CUST97418014,VÉRONIQUE DUMONT,TODD LLOYD,NaN,74.5,OSXK50207747,1.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN


,cust_id,trxn_amount,type,trxn_id,occ_wealth,occ_animal,occ_int,label
31146,CUST23957644,10555,deposit,KQDB81000112,1,0,1,1
145158,CUST95884667,270,withdrawal,QKKB49821994,0,0,0,0
162950,CUST58219673,5275,withdrawal,DVHO45420309,0,0,0,1
